In [ ]:
import pandas as pd
from pandasql import sqldf
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
pd.set_option('display.max_colwidth', 70)
pd.set_option('display.max_rows', None)

In [ ]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
#nltk.download('stopwords')
#nltk.download('wordnet')

In [ ]:
# setting a couple of workbook variables
text_source = "transcript"
tag = "opioid"

In [ ]:
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
df_human = pd.read_csv("data/classifications.csv")
df_ml = pd.read_csv("data/transcript_accuracy.csv")

In [ ]:
df_ml.head()

In [ ]:
df_human.head(1)

In [ ]:
# Todo? do we want to denormalize this, or get a list of distinct subjects

In [ ]:
df_dataset = pysqldf(f"""
SELECT dh.*, dml.{text_source}
FROM 
    df_human dh
JOIN
    df_ml dml
ON dh.ID = dml.id
    WHERE description is not NULL
    AND description != 'error code 224003'
""")

In [ ]:
len(df_dataset)

In [ ]:
df_dataset.head(2)

In [ ]:
#df_dataset.set_index("ID", inplace=True)

In [ ]:
df = pysqldf(f"""
SELECT 
    ID,
    subject,  
    {text_source},
    CASE
        WHEN subject LIKE '%{tag}%' THEN 1
        ELSE 0
    END AS category
FROM df_dataset
""")

In [ ]:
df.set_index("ID", inplace=True)

In [ ]:
df.head(1)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.5)

In [ ]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stop)),
    ('clf', RandomForestClassifier())
    ])

In [ ]:
X_train = text_clf['vect'].fit_transform(df_train[text_source])

In [ ]:
text_clf['vect'].get_feature_names_out()

In [ ]:
#X_train.todense()

In [ ]:
df_vector = pd.DataFrame(X_train.todense())
df_vector.columns = text_clf['vect'].get_feature_names_out()

In [ ]:
pd.set_option('display.max_rows', None)
#df_vector.T

In [ ]:
df_train.iloc[0][text_source]

In [ ]:
X_train = text_clf.fit(df_train[text_source], df_train['category'])

In [ ]:
feature_df = pd.DataFrame({"feature": text_clf['vect'].get_feature_names_out(), 
                           'importance': text_clf['clf'].feature_importances_})

In [ ]:
pysqldf("""
SELECT 
    *
FROM
    feature_df
ORDER BY
    importance DESC
""").head(10)

In [ ]:
feature_df.sort_values(by=['importance'], ascending=False)[:20]

In [ ]:
cross_val = cross_val_score(text_clf, df_test[text_source], df_test['category'], cv=4)

In [ ]:
cross_val

In [ ]:
y_proba = text_clf.predict_proba(df_test[text_source])
y_pred = text_clf.predict(df_test[text_source])

In [ ]:
# only list the first few, we'll build a dataframe on this later anyway
y_proba[:10]

In [ ]:
text_clf.classes_

In [ ]:
prob_not = [p[0] for p in y_proba]
prob_legal = [p[1] for p in y_proba]

In [ ]:
df_test['prob_no_match'] = prob_not
df_test['prob_match'] = prob_legal
df_test['y-pred'] = y_pred

In [ ]:
threshold = 0.5

y_adj = []

for p in prob_legal:
    if p <= threshold:
        y_adj.append(0)
    else:
        y_adj.append(1)
    
df_test['y_adj'] = y_adj

In [ ]:
df_test[['category', 'y-pred', 'y_adj', 'prob_no_match', 'prob_match']].head()

In [ ]:
tag_results = pysqldf(f"""
SELECT df_test.*, df.{text_source}, df.category
FROM df_test
JOIN df
ON df_test.ID = df.ID
""")

In [ ]:
tag_results.head()

In [ ]:
# let's look into where the model missed it
pysqldf("""
SELECT 
    *
FROM
    tag_results
WHERE 
    category != `y_adj`
""")

In [ ]:
y_pred = text_clf.predict(df_test[text_source])

In [ ]:
# everything below this line will vary based on the run

In [ ]:
precision, recall, fscore, train_support = precision_recall_fscore_support(df_test['category'], y_adj, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {} / fscore: {}'.format(
    round(precision, 3), round(recall, 3), (round((y_pred==df_test['category']).sum()/len(y_adj), 3)), round(fscore, 3)))

In [ ]:
print('f-score', 2 * ((precision * recall) / (precision + recall)))